# ADVI AEVB

The original Notebook from which my research was based

In [1]:
%matplotlib inline
import sys, os
# unfortunately I was not able to run it on GPU due to overflow problems
%env THEANO_FLAGS=device=cpu,floatX=float64
import theano

from collections import OrderedDict
from copy import deepcopy
import numpy as np
import pandas as pd
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt
import seaborn as sns
from theano import shared
import theano.tensor as tt
from theano.sandbox.rng_mrg import MRG_RandomStreams

import pymc3 as pm
from pymc3 import math as pmmath
from pymc3 import Dirichlet
from pymc3.distributions.transforms import t_stick_breaking
plt.style.use('seaborn-darkgrid')

import warnings
warnings.filterwarnings("ignore")

env: THEANO_FLAGS=device=cpu,floatX=float64


# Dataset

## Medium Articles (big) dataset

In [11]:
df_chunk = pd.read_csv(e"datasets/Medium_Clean.csv", chunksize=1000000)
    
chunk_list = []  # append each chunk df here 

# Each chunk is in df format
for chunk in df_chunk:  
    # perform data filtering 
   # chunk_filter = chunk_preprocessing(chunk)
    chunk_filter = chunk
    
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk_filter)
    
# concat the list into dataframe 
df_concat = pd.concat(chunk_list)

FileNotFoundError: [Errno 2] File b'.//datasets/Medium_Clean.csv' does not exist: b'.//datasets/Medium_Clean.csv'

In [12]:
df_concat.columns.values

NameError: name 'df_concat' is not defined

In [13]:
# for now just work with 'Title'
medium_stories = df_concat['Title']

# drop empty rows
medium_stories = medium_stories.dropna()

# convert to list
medium_stories = medium_stories.tolist()

medium_stories

NameError: name 'df_concat' is not defined

## Amazon fine food reviews 

In [14]:
df_chunk = pd.read_csv(r"../datasets/amazon-fine-food-reviews/Reviews.csv", chunksize=50000)
    
chunk_list = []  # append each chunk df here 

# Each chunk is in df format
for chunk in df_chunk:  
    # perform data filtering 
   # chunk_filter = chunk_preprocessing(chunk)
    chunk_filter = chunk
    
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk_filter)
    
# concat the list into dataframe 
df_concat = pd.concat(chunk_list)

print(df_concat.values)

[[1 'B001E4KFG0' 'A3SGXH7AUHU8GW' ... 1303862400 'Good Quality Dog Food'
  'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.']
 [2 'B00813GRG4' 'A1D87F6ZCVE5NK' ... 1346976000 'Not as Advertised'
  'Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".']
 [3 'B000LQOCH0' 'ABXLMWJIXXAIN' ... 1219017600 '"Delight" says it all'
  'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy t

we have reviews with scores and summaries, we are going to use the 'text' column of the review and the 'scores'

In [15]:
# scores
scores = df_concat['Score']

# reviews
reviews = df_concat['Text']
reviews.shape

(568454,)

# Clean data

In [17]:
# use if you using new dataset

# The number of words in the vocabulary
n_words = 1000

# input data
data_samples = reviews

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_words,
                                stop_words='english')

t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
feature_names = tf_vectorizer.get_feature_names()
print("done in %0.3fs." % (time() - t0))


Extracting tf features for LDA...
done in 99.385s.


In [3]:
# The number of words in the vocabulary
n_words = 1000

print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data
print("done in %0.3fs." % (time() - t0))

# Insert my datasets if required
# data_samples = ted_talks['description']
#data_samples = medium_stories

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_words,
                                stop_words='english')

t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
feature_names = tf_vectorizer.get_feature_names()
print("done in %0.3fs." % (time() - t0))

Loading dataset...
done in 1.987s.
Extracting tf features for LDA...
done in 2.649s.


In [4]:
# each doc is represented by a 1000-dimensional term freq-vector
# plt.plot(tf[:100, :].toarray().T);

# tf: term frequency doc matrix
tf.toarray().shape

(11314, 1000)

# train test split

In [20]:
# We split the whole documents into training and test sets. 
# The number of tokens in the training set is 480K. 
# Sparsity of the term-frequency document matrix is 0.025%, which implies almost all components in the term-frequency matrix is zero


n_samples_tr = round(tf.shape[0] * 0.7) # testing on 70%
n_samples_te = tf.shape[0] - n_samples_tr
docs_tr = tf[:n_samples_tr, :]
docs_te = tf[n_samples_tr:, :]
print('Number of docs for training = {}'.format(docs_tr.shape[0]))
print('Number of docs for testing = {}'.format(docs_te.shape[0]))


n_tokens = np.sum(docs_tr[docs_tr.nonzero()])
print('Number of tokens in training set = {}'.format(n_tokens))
print('Sparsity = {}'.format(
    len(docs_tr.nonzero()[0]) / float(docs_tr.shape[0] * docs_tr.shape[1])))

Number of docs for training = 7920
Number of docs for testing = 3394
Number of tokens in training set = 384502
Sparsity = 0.0255030303030303


# Log-likelihood of documents for LDA

In [21]:
def logp_lda_doc(beta, theta):
  
  """Returns the log-likelihood function for given documents.
  
  K : number of topics in the model
  V : number of words (size of vocabulary)
  D : number of documents (in a mini-batch)
  
  Parameters
  ----------
  beta : tensor (K x V)
      Word distribution.
  theta : tensor (D x K)
      Topic distributions for the documents.
  """
  
  def ll_docs_f(docs):
    
    dixs, vixs = docs.nonzero()
    vfreqs = docs[dixs, vixs]
    ll_docs = vfreqs * pmmath.logsumexp(
          tt.log(theta[dixs]) + tt.log(beta.T[vixs]), axis=1).ravel()
      
    # Per-word log-likelihood times no. of tokens in the whole dataset
    return tt.sum(ll_docs) / (tt.sum(vfreqs)+1e-9) * n_tokens

  return ll_docs_f

# LDA Model

In [32]:
n_topics = 20


# we have sparse dataset. It's better to have dence batch so that all words accure there
minibatch_size = 128

# defining minibatch
doc_t_minibatch = pm.Minibatch(docs_tr.toarray(), minibatch_size)
doc_t = shared(docs_tr.toarray()[:minibatch_size])

with pm.Model() as model:
    theta = Dirichlet('theta', a=pm.floatX((1.0 / n_topics) * np.ones((minibatch_size, n_topics))),
                   shape=(minibatch_size, n_topics), transform=t_stick_breaking(1e-9),
                   # do not forget scaling
                   total_size = n_samples_tr)
    beta = Dirichlet('beta', a=pm.floatX((1.0 / n_topics) * np.ones((n_topics, n_words))),
                 shape=(n_topics, n_words), transform=t_stick_breaking(1e-9))
        # Note, that we defined likelihood with scaling, so here we need no additional `total_size` kwarg
    doc = pm.DensityDist('doc', logp_lda_doc(beta, theta), observed=doc_t)

# Encoder

In [33]:
class LDAEncoder:
  """Encode (term-frequency) document vectors to variational means and (log-transformed) stds.
  """
  def __init__(self, n_words, n_hidden, n_topics, p_corruption=0, random_seed=1):
    rng = np.random.RandomState(random_seed)
    self.n_words = n_words
    self.n_hidden = n_hidden
    self.n_topics = n_topics
    self.w0 = shared(0.01 * rng.randn(n_words, n_hidden).ravel(), name='w0')
    



class LDAEncoder:
    """Encode (term-frequency) document vectors to variational means and (log-transformed) stds.
    """
    def __init__(self, n_words, n_hidden, n_topics, p_corruption=0, random_seed=1):
        rng = np.random.RandomState(random_seed)
        self.n_words = n_words
        self.n_hidden = n_hidden
        self.n_topics = n_topics
        self.w0 = shared(0.01 * rng.randn(n_words, n_hidden).ravel(), name='w0')
        self.b0 = shared(0.01 * rng.randn(n_hidden), name='b0')
        self.w1 = shared(0.01 * rng.randn(n_hidden, 2 * (n_topics - 1)).ravel(), name='w1')
        self.b1 = shared(0.01 * rng.randn(2 * (n_topics - 1)), name='b1')
        self.rng = MRG_RandomStreams(seed=random_seed)
        self.p_corruption = p_corruption

    def encode(self, xs):
        if 0 < self.p_corruption:
            dixs, vixs = xs.nonzero()
            mask = tt.set_subtensor(
                tt.zeros_like(xs)[dixs, vixs],
                self.rng.binomial(size=dixs.shape, n=1, p=1-self.p_corruption)
            )
            xs_ = xs * mask
        else:
            xs_ = xs

        w0 = self.w0.reshape((self.n_words, self.n_hidden))
        w1 = self.w1.reshape((self.n_hidden, 2 * (self.n_topics - 1)))
        hs = tt.tanh(xs_.dot(w0) + self.b0)
        zs = hs.dot(w1) + self.b1
        zs_mean = zs[:, :(self.n_topics - 1)]
        zs_rho = zs[:, (self.n_topics - 1):]
        return {'mu': zs_mean, 'rho':zs_rho}

    def get_params(self):
        return [self.w0, self.b0, self.w1, self.b1]

In [34]:
tf.shape
np.mean(np.sum(tf, axis=1))

48.641329326498145

In [35]:
encoder = LDAEncoder(n_words=n_words, n_hidden=100, n_topics=n_topics, p_corruption=0.0)
local_RVs = OrderedDict([(theta, encoder.encode(doc_t))])
local_RVs

OrderedDict([(theta,
              {'mu': Subtensor{::, :int64:}.0,
               'rho': Subtensor{::, int64::}.0})])

In [36]:
encoder_params = encoder.get_params()
encoder_params

[w0, b0, w1, b1]

# AEVB with ADVI

In [38]:
η = .1
s = shared(η)
def reduce_rate(a, h, i):
    s.set_value(η/((i/minibatch_size)+1)**.7)

with model:
    approx = pm.MeanField(local_rv=local_RVs)
    approx.scale_cost_to_minibatch = False
    inference = pm.KLqp(approx)
inference.fit(10000, callbacks=[reduce_rate], obj_optimizer=pm.sgd(learning_rate=s),
              more_obj_params=encoder_params, total_grad_norm_constraint=200,
              more_replacements={doc_t:doc_t_minibatch})

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 41 µs


Average Loss = 2.4674e+06: 100%|██████████| 10000/10000 [05:23<00:00, 30.91it/s]
Finished [100%]: Average Loss = 2.469e+06


In [13]:
print(approx)

Approximation{MeanFieldGroup[None, 19] & MeanFieldGroup[19980]}


# Extraction of characteristic words of topics based on posterior samples

In [14]:
def print_top_words(beta, feature_names, n_top_words=10):
    for i in range(len(beta)):
        print(("Topic #%d: " % i) + " ".join([feature_names[j]
            for j in beta[i].argsort()[:-n_top_words - 1:-1]]))


doc_t.set_value(docs_tr.toarray())
samples = pm.sample_approx(approx, draws=100)
beta_pymc3 = samples['beta'].mean(axis=0)

print_top_words(beta_pymc3, feature_names)

Topic #0: people god don think just know like say said time
Topic #1: edu use file key program information space available data ftp
Topic #2: just don like good think time year car know game
Topic #3: windows drive use card disk problem thanks does scsi like
Topic #4: know like don just does edu thanks good use think
Topic #5: 00 10 55 20 11 15 12 17 16 18
Topic #6: know like don just does edu thanks good use think
Topic #7: like know just don does edu thanks good new use
Topic #8: know like don just does edu thanks new think good
Topic #9: ax max g9v b8f a86 75u bhj 1t pl 145
Topic #10: like know don just does edu thanks good think new
Topic #11: like know don just does edu good thanks new think
Topic #12: know like just don does edu good thanks new think
Topic #13: db bh cs al bits cx just know like don
Topic #14: like just don know does edu good new thanks think
Topic #15: like don just know does edu good thanks people think
Topic #16: like don just know does edu good make com thank

# Compare to sklearn LDA implementations

In [15]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online', learning_offset=50.,
                                random_state=0)
%time lda.fit(docs_tr)
beta_sklearn = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]

print_top_words(beta_sklearn, feature_names)

CPU times: user 30.8 s, sys: 159 ms, total: 31 s
Wall time: 26 s
Topic #0: gun people state control law right crime police guns states
Topic #1: time like don question did book years just know think
Topic #2: db science line rules bh current int define title lines
Topic #3: key chip clipper keys use des encryption algorithm chips bit
Topic #4: edu com cs cx mail vs gm article uk send
Topic #5: does use point window problem value mean way different used
Topic #6: windows thanks know dos help does like problem using use
Topic #7: water bike effect design road dod paper inside turn hot
Topic #8: just don think like know people good going ve time
Topic #9: car price good power new used air ground sale wire
Topic #10: file available program information ftp edu files use list anonymous
Topic #11: ax max g9v pl b8f 75u a86 bhj 1t 34u
Topic #12: law government privacy security legal encryption fbi court private enforcement
Topic #13: color bit output memory video jpeg data image mode card
Topi

# Predictive distribution

In [16]:
def calc_pp(ws, thetas, beta, wix):
    """
    Parameters
    ----------
    ws: ndarray (N,)
        Number of times the held-out word appeared in N documents.
    thetas: ndarray, shape=(N, K)
        Topic distributions for N documents.
    beta: ndarray, shape=(K, V)
        Word distributions for K topics.
    wix: int
        Index of the held-out word

    Return
    ------
    Log probability of held-out words.
    """
    return ws * np.log(thetas.dot(beta[:, wix]))

def eval_lda(transform, beta, docs_te, wixs):
    """Evaluate LDA model by log predictive probability.

    Parameters
    ----------
    transform: Python function
        Transform document vectors to posterior mean of topic proportions.
    wixs: iterable of int
        Word indices to be held-out.
    """
    lpss = []
    docs_ = deepcopy(docs_te)
    thetass = []
    wss = []
    total_words = 0
    for wix in wixs:
        ws = docs_te[:, wix].ravel()
        if 0 < ws.sum():
            # Hold-out
            docs_[:, wix] = 0

            # Topic distributions
            thetas = transform(docs_)

            # Predictive log probability
            lpss.append(calc_pp(ws, thetas, beta, wix))

            docs_[:, wix] = ws
            thetass.append(thetas)
            wss.append(ws)
            total_words += ws.sum()
        else:
            thetass.append(None)
            wss.append(None)

    # Log-probability
    lp = np.sum(np.hstack(lpss)) / total_words

    return {
        'lp': lp,
        'thetass': thetass,
        'beta': beta,
        'wss': wss
    }

In [17]:
inp = tt.matrix(dtype='int64')
sample_vi_theta = theano.function(
    [inp],
    approx.sample_node(approx.model.theta, 100,  more_replacements={doc_t: inp}).mean(0)
)
def transform_pymc3(docs):
    return sample_vi_theta(docs)

In [18]:
%time result_pymc3 = eval_lda(transform_pymc3, beta_pymc3, docs_te.toarray(), np.arange(100))
print('Predictive log prob (pm3) = {}'.format(result_pymc3['lp']))

CPU times: user 1min 24s, sys: 3.51 s, total: 1min 27s
Wall time: 1min 22s
Predictive log prob (pm3) = -6.078433976876863


In [19]:
def transform_sklearn(docs):
    thetas = lda.transform(docs)
    return thetas / thetas.sum(axis=1)[:, np.newaxis]

%time result_sklearn = eval_lda(transform_sklearn, beta_sklearn, docs_te.toarray(), np.arange(100))
print('Predictive log prob (sklearn) = {}'.format(result_sklearn['lp']))

CPU times: user 3min 31s, sys: 2.21 s, total: 3min 33s
Wall time: 3min 5s
Predictive log prob (sklearn) = -5.991822468667646
